# Segmenting and Clustering Neighborhoods in Toronto

<h3>Introduction</h3>
<p>
This is an assignment for the Introduction to Artifical Intelligence course (SOFE 3720U). Within this we will be explore how to segement and cluster the neighborhoods in Toronto. 
</p>

<h3>Import Statements</h3>

In [2]:
from dotenv import load_dotenv
from dotenv import dotenv_values

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import geojson

import requests
from pandas import json_normalize

import folium

from bs4 import BeautifulSoup as bs

print('Libraries Imported')

Libraries Imported


<h3>Week 1 - Foursquare API</h3>
<p>
Within this section we will be using the Foursquare API to find latitude, longitude, and venues within the Toronto area.
</p>

<h4>
Setting up Foursquare API
</h4>

In [4]:
#Import the hidden values within the .env file, these values are keys used to access API
config = dotenv_values(".env")
#Assign url variable to initialize API
url = "https://api.foursquare.com/v3/places/nearby"
#Assign header which will allow us to access the website by passing through keys 
headers = {"Accept": "application/json","Authorization": config["API_KEY"]}
#Create the request statement which allows to freely use API
response = requests.request("GET", url, headers=headers)

#Initialize and define findNearbyVenues function
def findNearbyVenues(location, categories, limit):
    #Assign url variable which is based on the parameters passed through function call
    url = "https://api.foursquare.com/v3/places/search?" + "categories=" + categories + "&near=" + str(location[0]) + "%2C" + str(location[1]) + "&limit=" + limit
    #Create the response statement from requesting from API
    response = requests.request("GET", url, headers=headers)
    #Return the result if the code was successful
    if(response.status_code == 200):
        return response.json()
    #Return false if the code didn't work
    else:
        return False
    
print('API initialize and custom function created')

API initialize and custom function created


<h4>
Creating dataframe using the function created to use Foursquare API
</h4>

In [11]:
#Assign lat and long for Toronto
latitude = 43.6532 
longitude = -79.3832
#Assign result variable use the custom function with the parameters below
results = findNearbyVenues(location = ["Toronto", "ON"], categories="17000", limit="50")
#Normalize the results to be able to changed
venuesdf = json_normalize(results['results'], max_level=3)
#Drop unnecessary columns
venuesdf.drop(venuesdf.columns[[0,1,2,3,5,8,9,10,11,12,13,17,18,19,20,22]], axis=1, inplace=True)
#Display first five rows
venuesdf.head()

,name,geocodes.main.latitude,geocodes.main.longitude,location.locality,location.neighborhood,location.postcode,related_places.parent.name
0,Fiesta Farms,43.668877,-79.420664,Toronto,[Christie Pitts],M6G 3B6,NaN
1,Yorkdale,43.725902,-79.453193,Toronto,[Downsview],M6A 2T9,NaN
2,Uniqlo ユニクロ,43.726446,-79.450564,Toronto,"[Lawrence Heights, Toronto, ON]",M6A 2T9,Yorkdale
3,Loblaws,43.661965,-79.379811,Toronto,NaN,M5B 1J2,NaN
4,CF Toronto Eaton Centre,43.653807,-79.380056,Toronto,[Eaton Centre],M5B 2H1,NaN


In [12]:
#Create the map which is based on the coordinates for Toronto
map_creation = folium.Map(location=[latitude, longitude], zoom_start=10)
#Display Map
map_creation

<h3>Week 2 - Prepare your data</h3>
<p>
Within this section we will be using the provide source to create a large dataframe which contains the neccessary information for the choosen correlations
</p>

In [20]:
"""url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'
temp = requests.get(url)
data = temp.text
soup = bs(data,'html.parser')
wiki = soup.find('table')
df = pd.read_html(str(wiki))[0]
df.head()"""

"url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'\ntemp = requests.get(url)\ndata = temp.text\nsoup = bs(data,'html.parser')\nwiki = soup.find('table')\ndf = pd.read_html(str(wiki))[0]\ndf.head()"

In [21]:
"""df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.index = range(len(df))
df.head()"""

"df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)\ndf.index = range(len(df))\ndf.head()"

In [17]:
'''with open('Crime_Rates.geojson') as f:
    data = geojson.load(f)
df2=pd.json_normalize(data["features"])
df2=df2.drop(columns=["type", "geometry.type", "geometry.coordinates", "properties.OBJECTID", "properties.Shape__Area", "properties.Shape__Length"])
df2.head()'''

'with open(\'Crime_Rates.geojson\') as f:\n    data = geojson.load(f)\ndf2=pd.json_normalize(data["features"])\ndf2=df2.drop(columns=["type", "geometry.type", "geometry.coordinates", "properties.OBJECTID", "properties.Shape__Area", "properties.Shape__Length"])\ndf2.head()'